<a href="https://colab.research.google.com/github/dannsb/neural-network/blob/main/Next_Word_Prediction(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import libs

In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense,Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
file = open("data.txt","r",encoding="utf8")

### pre proccessing on text data

In [3]:
lines = []
for i in file:
  lines.append(i)

In [4]:
data=""
for i in lines:
    data = ' '.join(lines)
data = data.replace("\n","").replace("\r","").replace("ufeff","").replace('“','').replace('”','')

In [5]:
data = data.split()
data = " ".join(data)

Tokenizer: creates the vocabulary index based on word frequency

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

Pickle “serializes” the object first before writing it to a file. Pickling is a way to convert a Python object (list, dictionary, etc.) into a character stream.

In [7]:
pickle.dump(tokenizer, open("token.pkl","wb"))

In [8]:
sequenceData = tokenizer.texts_to_sequences([data])[0]

In [9]:
sequence=[]
for i in range(3, len(sequenceData)):
    words = sequenceData[i-3:i+1]
    sequence.append(words)

In [10]:
sequence = np.array(sequence)

In [11]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [12]:
X = np.array(X)
y = np.array(y)

In [13]:
vocabSize = len(tokenizer.word_index)+1
y = to_categorical(y, vocabSize)

### add model & fit

In [42]:
model = Sequential()
model.add(Embedding(input_dim=vocabSize, output_dim=10))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocabSize, activation="softmax"))
model.build(input_shape=(vocabSize,None))

In [43]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("nextword.keras", monitor='loss', verbose=1,
    save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy')

In [44]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ (8200, None, 10)            │          82,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ (8200, None, 1000)          │       4,044,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (8200, 1000)                │       8,004,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (8200, 1000)                │       1,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (8200, 8200)                │       8,208,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,339,200 (81.40 MB)

 Trainable params: 21,339,200 (81.40 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.fit(X, y, epochs=2, batch_size=64, callbacks=[checkpoint]) #NOTE: epochs should be more

Epoch 1/2
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 6.6633
Epoch 1: loss improved from inf to 6.35357, saving model to nextword.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 47s 24ms/step - loss: 6.6631
Epoch 2/2
1653/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.8033
Epoch 2: loss improved from 6.35357 to 5.76899, saving model to nextword.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 39s 24ms/step - loss: 5.8033


### predict function
besed on keras model file generated by RNN(LSTM).in my example: **nextword.keras**

In [18]:
from tensorflow.keras.models import load_model

model = load_model('nextword.keras')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def predict_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds=np.argmax(model.predict(sequence))
    predicted_word = ""
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    return predicted_word


predict_word(model, tokenizer, "A Scandal in")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step


'the'